# Optics Matching & Slow Extraction

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xtrack as xt

%config InlineBackend.figure_format = "retina"

### Load Model and Set Initial Strengths

In [ ]:
env = xt.Environment.from_json("pimm.json")
ring = env["ring"]

In [ ]:
# Assigned 'randomly' to get a computable Twiss
env["kqfa"] = 0.01
env["kqfb"] = 0.01
env["kqd"] = -0.02

In [ ]:
twiss = ring.twiss4d(compute_chromatic_properties=False)
twiss.plot()

### Match the Tunes

In [ ]:
opt_tune = ring.match(
    solve=False,  # <- prepare the match but do not run
    vary=[
        xt.Vary("kqfa", limits=(0, 10), step=1e-3),
        xt.Vary("kqfb", limits=(0, 10), step=1e-3),
        xt.Vary("kqd", limits=(-10, 0), step=1e-3),
    ],
    targets=[  # Horizontal tune close to 3rd order resonance
        xt.TargetSet(qx=1.665, qy=1.72, tol=1e-4),
    ],
    compute_chromatic_properties=False,
    method="4d",
)

In [ ]:
opt_tune.target_status()

In [ ]:
opt_tune.run_jacobian(30)

In [ ]:
opt_tune.target_status()

In [ ]:
opt_tune.vary_status()

In [ ]:
# Inspect the optics
twiss = ring.twiss4d()
plot = twiss.plot()
plot.ylim(left_hi=40, right_lo=-20, right_hi=20, lattice_hi=1.5, lattice_lo=-7)

### Add Dispersion Constraint at Center of the Long Straight Sections

In [ ]:
opt_disp = opt_tune.clone(  # clone to keep previous constraints!
    add_targets=[
        xt.TargetSet(dx=0, at="mid.lss.0"),
        xt.TargetSet(dx=0, at="mid.lss.1"),
    ]
)

In [ ]:
opt_disp.target_status()

In [ ]:
opt_disp.run_simplex(100)  # Nelder-Mead optimisation algorithm

In [ ]:
# Inspect the optics
twiss = ring.twiss4d()
plot = twiss.plot()
plot.ylim(left_hi=40, right_lo=-20, right_hi=20, lattice_hi=1.5, lattice_lo=-7)

## Correct Chromaticity

In [ ]:
opt_chrom = ring.match(
    solve=True,
    method="4d",
    vary=xt.VaryList(["ksf", "ksd"], step=1e-3),
    targets=xt.TargetSet(dqx=-0.01, dqy=-0.01, tol=1e-3),
)

In [ ]:
opt_chrom.run_ls_dogbox(30)  # Least squares, dobgox algorithm

In [ ]:
opt_chrom.target_status()

## Observe Phase Space Deformation when Exciting the Resonance

In [ ]:
# Generate 20 particles on the x axis
x_gen = np.linspace(0, 0.5e-2, 20)
parts = ring.build_particles(x=x_gen, px=0, y=0, py=0, zeta=0, delta=0)

In [ ]:
# Set extraction sextupole
env["kse"] = 6

# Track 1000 turns
ring.track(parts, num_turns=1000, turn_by_turn_monitor=True)
record = ring.record_last_track

In [ ]:
# Plot turn-by-turn data
plt.figure()
plt.plot(record.x.T, record.px.T, ".", markersize=1, color="C0")
plt.xlim(-2e-2, 2e-2)
plt.ylim(-2e-3, 2e-3)
plt.xlabel(r"$x$ [m]")
plt.ylabel(r"$p_x$")
plt.title("Horizontal Phase Space")
plt.subplots_adjust(left=0.15)
plt.show()

### Exports Settings

In [ ]:
# Extract the strengths from optimisers
quad_strengths: dict[str, float] = opt_disp.get_knob_values()
sext_strengths: dict[str, float] = opt_chrom.get_knob_values()

In [ ]:
# Merge dictionaries and save to file
strengths = quad_strengths | sext_strengths
xt.json.dump(strengths, "pimm_strengths.json")

---